# **Tensorflow basics**


# **Initialization**

In [2]:
#importing packages and initializing the tensor

import tensorflow as tf
import numpy as np

rows=6
cols=5
source=tf.random.uniform(shape=[rows,cols],minval=0, maxval=10,dtype=tf.dtypes.int32)
print(source.numpy())

[[9 6 6 3 4]
 [1 5 5 5 3]
 [0 7 8 0 4]
 [8 3 2 9 6]
 [4 8 9 4 5]
 [5 2 7 5 1]]


# **Task 1:Top K**

In [0]:
#function to find the top-k values in each row and their indices
@tf.function
def topKValues(x):
  values, positions = tf.math.top_k(x, k=1, sorted=True, name=None)
  return (values,positions)

In [0]:
values,positions = topKValues(source)
print (values.numpy())
print(positions.numpy())

[[9]
 [7]
 [9]
 [8]
 [8]
 [7]]
[[2]
 [4]
 [2]
 [4]
 [1]
 [0]]


# **Task 2:Argmax**

In [0]:
 #function find the argmax value of each row and replace it by 1 and remaining by 0
 @tf.function
 def argmax(source):
  # get indices of argmax
  k = 1       #each row will have 1 argmax
  argMax_inds = tf.reshape(tf.math.argmax(source,axis=1,output_type=tf.dtypes.int32), [rows,1]) 

  num_rows = tf.shape(source)[0]
  row_range = tf.range(num_rows)
  row_tensor = tf.tile(row_range[:,None], (1, k))

  argMax_row_col_indices = tf.stack([row_tensor, argMax_inds], axis=2)


  updates = tf.ones([num_rows, k], dtype=tf.int32)

  # build the mask
  zero_mask = tf.scatter_nd(argMax_row_col_indices, updates, [num_rows, cols])

  
  NUMBER = 0
  zeroed = zero_mask/zero_mask
  zeroed = tf.where(tf.math.is_nan(zeroed), tf.ones_like(zeroed) * NUMBER, zeroed)
  return (zeroed)

In [0]:
solution = argmax(source)
print (solution.numpy())

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


# **Task 3: topK_Zeros**

In [0]:
 #function to find the top-k values of each row and replace remaining values with 0
 @tf.function
 def topK(source):
  # get indices of top k
  k = 1
  top_k, top_k_inds = tf.math.top_k(source, k=1, sorted=False, name=None)
  # indices are only columns, we will stack 
  # it so the row indice is also there and
  # make tensor of row numbers ie.
  # [[0, 0],
  #  [1, 1],
  #  ...
  num_rows = tf.shape(source)[0]
  row_range = tf.range(num_rows)
  row_tensor = tf.tile(row_range[:,None], (1, k))

  # stack along the final dimension, as this is what
  # scatter_nd uses as the indices
  top_k_row_col_indices = tf.stack([row_tensor, top_k_inds], axis=2)

  # to mask off everything, we will multiply the top_k by
  # 1. so all the updates are just 1
  updates = tf.ones([num_rows, k], dtype=tf.int32)

  # build the mask
  zero_mask = tf.scatter_nd(top_k_row_col_indices, updates, [num_rows, cols])

  #with tf.Session() as sess:
  zeroed = source*zero_mask
  #print(zeroed.numpy())
  return(zeroed)

In [0]:
solution = topK(source)
print(solution.numpy())

[[8 0 0 0 0]
 [0 8 0 0 0]
 [7 0 0 0 0]
 [8 0 0 0 0]
 [7 0 0 0 0]
 [0 0 0 6 0]]


# **Task 4:Exponential Moving Avg**

In [0]:
#function to find the exponential moving average
@tf.function
def movingAvg(alpha,tsor):
  oput = tsor
  # stacking positional index with the original value.
  elems = tf.stack([tf.range(tsor.shape[0],dtype=tf.float32),tsor], axis = -1)
  fn = lambda x: oput[tf.cast(x[0], tf.int32)-1]*alpha + (1-alpha)*x[1] if x[0] > 0 else oput[0]
  oput = tf.map_fn(fn, elems)
  return oput

In [0]:
rows = 10
input =tf.random.uniform(shape=[rows], minval=1, maxval=10, dtype=tf.float32)
print("Input Array: ",input.numpy())
print("Moving Avg;",movingAvg(.001,input).numpy())

Input Array:  [5.1156244 9.715345  3.126422  4.0886645 2.438343  9.730957  1.0502034
 3.0675154 8.663475  6.6983423]
Moving Avg; [5.1156244 9.710746  3.1330109 4.0877028 2.4399934 9.723664  1.0588841
 3.065498  8.65788   6.700308 ]


# **Task 5:Return Last Position**

In [0]:
#print the last position value of the exponential moving average
def movingAvgInd(a,input):
  A = movingAvg(a,input)
  return (A.numpy()[-1])

In [0]:
inp_1D =tf.random.uniform(shape=[rows], minval=1, maxval=10, dtype=tf.float32)
print("Input Array: ",inp_1D.numpy())
movingAvgInd(.001,inp_1D)

Input Array:  [1.6783918 8.263931  3.5048451 1.1005732 5.335116  1.908608  3.743784
 8.911583  2.8067274 3.9882598]


3.9870782

# **Task 6: Swap**

In [0]:
 def  Swap(row_size):
  rows = row_size
  inp =tf.random.uniform(shape=[rows], minval=1, maxval=10, dtype=tf.int32)
  y = tf.random.uniform(shape=[rows], minval=1, maxval=10, dtype=tf.int32)
  z = tf.random.uniform(shape=[rows], minval=1, maxval=10, dtype=tf.int32)
  print("x:",inp.numpy())
  print("y:",y.numpy())
  print("z:",z.numpy())
  tsor = inp 
  oput = tsor
  # stacking positional index with the original value.
  fn = lambda x: y[x] if tf.math.floormod(inp[x], 2).numpy() == 0 else z[x]

  oput = tf.map_fn(fn, tf.range(tsor.shape[0]).numpy())
  print("Input :", inp.numpy())
  print("Output :", oput.numpy())
  return (oput)

In [0]:
print(Swap(10))

x: [6 1 1 1 1 6 4 3 6 9]
y: [5 6 2 5 6 2 7 6 3 2]
z: [8 2 8 4 6 9 8 5 3 3]
Input : [6 1 1 1 1 6 4 3 6 9]
Output : [5 2 8 4 6 2 7 5 3 3]
tf.Tensor([5 2 8 4 6 2 7 5 3 3], shape=(10,), dtype=int32)


# **Task 7: UnkownShape**

Test Case Generation

In [0]:
rows = 10
cols = 34
height = 20
d4 = 0
d5 = 0
#inp =tf.random.uniform(shape=[rows,cols,height,d4,  ], minval=1, maxval=10, dtype=tf.int32)
inp =tf.random.uniform(shape=[rows,cols,], minval=1, maxval=10, dtype=tf.int32)
tsor = inp 
print("Output Should be: ",tf.shape(tsor).numpy()[-1])

Output Should be:  34


In [0]:
def LastDim(tensor):
  if(tf.shape(tensor).numpy()[-1] > 100):
    return 100
  elif(tf.shape(tensor).numpy()[-1] > 44 and tf.shape(tensor).numpy()[-1] <= 100):
    return 12
  else:
    return 0

In [0]:
LastDim(inp)

0

# **Task 8: CountCondition**

Global Variable Declaration


In [0]:
count_i = 0
count_j = 0
count_k = 0

In [0]:
#@tf.function
def CountCheck(tensor):
  global count_i
  global count_j
  global count_k
  if(tf.shape(tensor).numpy()[-1] > 100):
    count_i+= 1
    return 100
  elif(tf.shape(tensor).numpy()[-1] > 44 and tf.shape(tensor).numpy()[-1] <= 100):
    count_j+= 1
    return 12
  else:
    count_k+= 1
    return 0

In [0]:
rows = 10
cols = 34
height = 200
d4 = 0
d5 = 0
#inp =tf.random.uniform(shape=[rows,cols,height,d4,  ], minval=1, maxval=10, dtype=tf.int32)
inp =tf.random.uniform(shape=[rows,cols,height], minval=1, maxval=10, dtype=tf.int32)
tsor = inp 
print("Output Should be: ",tf.shape(tsor).numpy()[-1])
CountCheck(inp)
print(count_i,count_j,count_k)

Output Should be:  200
4 4 4


# **Task 9: TensorCalculation**

In [0]:
import numpy as np
rows = 5
x = tf.random.uniform(shape=[rows,1], minval=1, maxval=10, dtype=tf.int32)
y = tf.random.uniform(shape=[rows], minval=1, maxval=10, dtype=tf.int32)
print(y.numpy())
a = tf.constant([1,rows])
b = tf.tile(x, a)
print(b.numpy())
tf.math.subtract(b, y).numpy()

# **Task 10:**

In [5]:
batch = 128
time = 10
features = 5
h = tf.random.uniform(shape=[batch,time,features], minval=1, maxval=10, dtype=tf.int32)
s = tf.random.uniform(shape=[batch,features], minval=1, maxval=10, dtype=tf.int32)
alpha = tf.random.uniform(shape=[batch,time], minval=1, maxval=10, dtype=tf.int32)
#for i in range(time):
# alpha[:, i] = tf.tensordot(h[:, i], s, axes = 0)
#print(alpha)
# stacking positional index with the original value.
#elems = tf.stack([tf.range(tsor.shape[0],dtype=tf.float32),tsor], axis = -1)
fn = lambda x: tf.tensordot(h[:, x], s, axes = 0)
alpha = tf.map_fn(fn, tf.range(time))
print(alpha)

tf.Tensor(
[[[[[14 63 14 42  7]
    [35 28 56  7 63]
    [35 49 63 49 42]
    ...
    [49 63  7  7 35]
    [ 7 42 35 49 56]
    [42 14 42 63 28]]

   [[12 54 12 36  6]
    [30 24 48  6 54]
    [30 42 54 42 36]
    ...
    [42 54  6  6 30]
    [ 6 36 30 42 48]
    [36 12 36 54 24]]

   [[ 8 36  8 24  4]
    [20 16 32  4 36]
    [20 28 36 28 24]
    ...
    [28 36  4  4 20]
    [ 4 24 20 28 32]
    [24  8 24 36 16]]

   [[16 72 16 48  8]
    [40 32 64  8 72]
    [40 56 72 56 48]
    ...
    [56 72  8  8 40]
    [ 8 48 40 56 64]
    [48 16 48 72 32]]

   [[16 72 16 48  8]
    [40 32 64  8 72]
    [40 56 72 56 48]
    ...
    [56 72  8  8 40]
    [ 8 48 40 56 64]
    [48 16 48 72 32]]]


  [[[ 8 36  8 24  4]
    [20 16 32  4 36]
    [20 28 36 28 24]
    ...
    [28 36  4  4 20]
    [ 4 24 20 28 32]
    [24  8 24 36 16]]

   [[ 4 18  4 12  2]
    [10  8 16  2 18]
    [10 14 18 14 12]
    ...
    [14 18  2  2 10]
    [ 2 12 10 14 16]
    [12  4 12 18  8]]

   [[ 2  9  2  6  1]
    [ 5  4  8 